# Run Delphes and extract observables

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os
import math
from collections import OrderedDict

from madminer.delphes import DelphesProcessor
from madminer.sampling import combine_and_shuffle
from madminer.utils.particle import MadMinerParticle

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')


In [2]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [3]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'

## Observables and cuts

In [4]:
def calculate_mt(leptons, photons, jets, met):
    # Particles
    if len(leptons) < 1:
        raise RuntimeError()
    
    l = leptons[0]
    
    # Transverse mass and Delta
    # mt = ((l.pt + met.pt)**2 - (l+met).pt**2)**0.5
    cos_delta_phi = np.cos(l.phi() - met.phi())
    mt = (2 * l.pt * met.pt * (1. - cos_delta_phi))**0.5
    
    return mt

In [5]:
def calculate_phi(leptons, photons, jets, met):
    # Parameters
    mw = 80.4
    
    # Particles
    if len(leptons) < 1 or len(photons) < 1:
        raise RuntimeError()
    
    l = leptons[0]
    a = photons[0]
    
    # Transverse mass and Delta
    mt = calculate_mt(leptons, photons, jets, met)
    deltasq = 0.
    if met.pt > 0. and l.pt > 0.:
        deltasq = (mw**2 - mt**2) / (2. * met.pt * l.pt)
    
    # v reconstruction, "normal" case
    if deltasq > 0.:
        # Two solutions
        temp = np.log(1 + deltasq**0.5 * (2 + deltasq)**0.5 + deltasq)
        eta_v_plus = l.eta + temp
        eta_v_minus = l.eta - temp
        
        # Randomly select one of them
        dice = np.random.rand()
        if dice > 0.5:
            eta_v = eta_v_plus
        else:
            eta_v = eta_v_minus
            
    # v reconstruction, "other" case
    else:
        eta_v = l.eta
        
    # v particle
    v = MadMinerParticle()
    v.setptetaphim(met.pt, eta_v, met.phi(), 0.)
    
    # W and Wgamma reconstruction
    w = l + v
    vv = w + a
    
    # Boost into VV frame
    v_ = v.boost(vv.boostvector)
    l_ = l.boost(vv.boostvector)
    a_ = a.boost(vv.boostvector)
    w_ = w.boost(vv.boostvector)
    r_ = vv # vv.boost(vv.boostvector)

    # Calculate axes of "special frame" (1708.07823)
    z_ = w_.vector.unit()
    x_ = (r_.vector - z_ * r_.vector.dot(z_)).unit()
    y_ = z_.cross(x_)
    
    # Calculate x and y components of lepton wrt special x_, y_, z_ system
    lx_ = l_.vector.dot(x_)
    ly_ = l_.vector.dot(y_)
    
    # Calculate phi
    phi = math.atan2(ly_, lx_)
    
    return phi
    

In [6]:
def setup_observables(delphesprocessor):
    delphesprocessor.observables = OrderedDict()
    delphesprocessor.observables_required = OrderedDict()
    delphesprocessor.observables_defaults = OrderedDict()
        
    # Default observables
    delphesprocessor.add_default_observables(
        n_leptons_max=1,
        n_photons_max=1,
        n_jets_max=1,
        include_charge=False,
        include_numbers=False,
        include_met=True
    )

    # Correlations with MET
    for s1, i1 in [('l', 0), ('a', 0)]:
        delphesprocessor.add_observable(
            'deltaphi_{}{}_met'.format(s1, i1+1),
            '{}[{}].phi() - met.phi()'.format(s1, i1),
            required=True
        )

    # Reconstructed W
    delphesprocessor.add_observable(
        'm_l1_met',
        '(l[0] + met).m',
        required=True
    )
    delphesprocessor.add_observable(
        'pt_l1_met',
        '(l[0] + met).pt',
        required=True
    )

    # Selected correlations between particles
    for s1, i1, s2, i2 in [('l', 0, 'a', 0)]:
        delphesprocessor.add_observable(
            'm_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            '({}[{}] + {}[{}]).m'.format(s1, i1, s2, i2),
            required=True
        )
        delphesprocessor.add_observable(
            'deltaeta_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            '{}[{}].eta - {}[{}].eta'.format(s1, i1, s2, i2),
            required=True
        )
        delphesprocessor.add_observable(
            'deltaphi_{}{}_{}{}'.format(s1, i1+1, s2, i2+1),
            '{}[{}].phi() - {}[{}].phi()'.format(s1, i1, s2, i2),
            required=True
        )

    # Wgamma system
    delphesprocessor.add_observable(
        'm_a1_l1_met',
        '(a[0] + l[0] + met).m',
        required=True
    )
    delphesprocessor.add_observable(
        'pt_a1_l1_met',
        '(a[0] + l[0] + met).pt',
        required=True
    )
    
    # mT(W)
    delphesprocessor.add_observable_from_function(
        'mt',
        calculate_mt,
        required=True
    )
    
    # Resurection phi
    delphesprocessor.add_observable_from_function(
        'phi_resurrection',
        calculate_phi,
        required=True
    )


In [7]:
def setup_cuts(delphesprocessor, use_tight_cuts=False):
    delphesprocessor.cuts = []
    delphesprocessor.cuts_default_pass = []
    
    if use_tight_cuts:
        delphesprocessor.add_cut(
            'pt_a1 >= 300.'
        )
        delphesprocessor.add_cut(
            'pt_l1 >= 80.'
        )
        delphesprocessor.add_cut(
            'et_miss >= 80.'
        )
        delphesprocessor.add_cut(
            '(deltaphi_l1_a1**2 + deltaeta_l1_a1**2)**0.5 >= 3.'
        )
        delphesprocessor.add_cut(
            'eta_l1**2 < 2.4**2'
        )
        
    else:
        delphesprocessor.add_cut(
            'et_miss >= 10.'
        )

## Main loop

In [11]:
n_runs_per_benchmark = 18  # Number of run_cards

In [12]:
run = 0

for i_card in range(n_runs_per_benchmark):
    logging.info('Starting analysis of runs for card {}'.format(i_card))
            
    # Load setup
    dp = DelphesProcessor(sample_dir + 'setup.h5', debug=True)
    
    # Load events
    for benchmark in ['sm']:
        run += 1
        run_str = str(run)
        if len(run_str) < 2:
            run_str = '0' + run_str

        if run < 15:
            dp.add_hepmc_sample(
                mg_process_dir + 'Events/run_{}/tag_2_pythia8_events.hepmc.gz'.format(run_str),
                sampled_from_benchmark=benchmark
            )
        else:
            dp.add_hepmc_sample(
                mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc.gz'.format(run_str),
                sampled_from_benchmark=benchmark
            )
        
    # Run Delphes
    dp.run_delphes(
        delphes_directory=delphes_dir,
        delphes_card=card_dir + 'delphes_card.dat',
        log_directory=log_dir,
        initial_command='source activate python2',
    )
    
    # Set up observables and cuts
    setup_observables(dp)
    setup_cuts(dp)
    
    # Analyse Delphes sample
    dp.analyse_delphes_samples(delete_delphes_files=True)
        
    # Save
    dp.save(sample_dir + 'samples_{}.h5'.format(i_card))


22:57  Starting analysis of runs for card 0
22:57  Benchmarks found in MadMiner file: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
22:57  Adding HepMC sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_2_pythia8_events.hepmc.gz
22:57  Calling gunzip -k /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_2_pythia8_events.hepmc.gz > None
22:57  Found weight labels in HEPMC file: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
22:57  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_2_pythia8_events.hepmc.gz
22:57  Unzipping /Users/johannb

23:07    First 10 values for observable phi_resurrection:
[ 0.25903842 -0.46128309 -2.25762224 -1.73212685         nan -3.09653875
 -1.42050134 -2.46948499         nan  0.22756084]
23:07    150000 / 150000 events pass required observable et_miss
23:07    150000 / 150000 events pass required observable phi_miss
23:07    150000 / 150000 events pass required observable e_visible
23:07    150000 / 150000 events pass required observable eta_visible
23:07    101446 / 150000 events pass required observable deltaphi_l1_met
23:07    127057 / 150000 events pass required observable deltaphi_a1_met
23:07    101446 / 150000 events pass required observable m_l1_met
23:07    101446 / 150000 events pass required observable pt_l1_met
23:07    86004 / 150000 events pass required observable m_l1_a1
23:07    86004 / 150000 events pass required observable deltaeta_l1_a1
23:07    86004 / 150000 events pass required observable deltaphi_l1_a1
23:07    86004 / 150000 events pass required observable m_a1_l1_met

23:16    First 10 values for observable eta_j1:
[0.         0.         0.         0.         2.01580715 2.31036997
 0.         0.         0.         0.        ]
23:16    First 10 values for observable phi_j1:
[ 0.          0.          0.          0.          1.14028227 -2.77237988
  0.          0.          0.          0.        ]
23:16    First 10 values for observable deltaphi_l1_met:
[ 3.72873855 -1.91701633         nan  0.27791488  3.94952428 -0.7358526
 -2.05209543  1.86582237  1.63033569  2.02273038]
23:17    First 10 values for observable deltaphi_a1_met:
[ 0.76546359 -3.35228023  2.71280984 -3.17457557  1.03986955  2.56287069
  1.47133552  3.50933456  4.24770272  4.28107405]
23:17    First 10 values for observable m_l1_met:
[ 84.79766204  82.44571336          nan  61.88095599  70.35458452
  69.47456526  72.63630653  65.68993149  77.84980402 118.37883023]
23:17    First 10 values for observable pt_l1_met:
[ 57.95454522  47.49851379          nan 104.13624104  34.37989912
 163.4202

23:26    First 10 values for observable phi_miss:
[-2.20155859  2.83424854 -1.09290993  2.12854362 -0.72522628 -0.34101546
 -2.12754226 -2.93995404 -1.596506   -3.12427759]
23:26    First 10 values for observable e_visible:
[326.30325759 126.38843178 147.3119326  826.9687683    0.
 294.64612252 291.34975071 227.12116106 121.13075065 694.62957808]
23:26    First 10 values for observable eta_visible:
[-8.51808906e-01  8.12789083e-01  1.26119328e+00  2.38828901e+00
 -1.00000000e+11  8.79842639e-01 -1.00556064e+00  1.92302159e+00
 -1.60677755e+00  1.98417438e+00]
23:26    First 10 values for observable e_l1:
[117.77667236  46.84854507  38.63467026  54.27111053   0.
 104.28588104  94.00566101  21.78909492  23.35160255   0.        ]
23:27    First 10 values for observable pt_l1:
[163.15162879  63.19421589  73.6559663  325.6045877    0.
 147.32306126 145.67487535  37.89142092  60.56537533   0.        ]
23:27    First 10 values for observable eta_l1:
[-0.85180891  0.81278908  1.26119328  2.477

23:37  Adding required observable et_miss = met.pt
23:37  Adding required observable phi_miss = met.phi()
23:37  Adding required observable e_visible = visible.e
23:37  Adding required observable eta_visible = visible.eta
23:37  Adding optional observable e_l1 = l[0].pt with default 0.0
23:37  Adding optional observable pt_l1 = l[0].e with default 0.0
23:37  Adding optional observable eta_l1 = l[0].eta with default 0.0
23:37  Adding optional observable phi_l1 = l[0].phi() with default 0.0
23:37  Adding optional observable e_a1 = a[0].pt with default 0.0
23:37  Adding optional observable pt_a1 = a[0].e with default 0.0
23:37  Adding optional observable eta_a1 = a[0].eta with default 0.0
23:37  Adding optional observable phi_a1 = a[0].phi() with default 0.0
23:37  Adding optional observable e_j1 = j[0].pt with default 0.0
23:37  Adding optional observable pt_j1 = j[0].e with default 0.0
23:37  Adding optional observable eta_j1 = j[0].eta with default 0.0
23:37  Adding optional observable

...

01:57  Deleting /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_15/tag_1_pythia8_events.hepmc
01:57  Adding required observable et_miss = met.pt
01:57  Adding required observable phi_miss = met.phi()
01:57  Adding required observable e_visible = visible.e
01:57  Adding required observable eta_visible = visible.eta
01:57  Adding optional observable e_l1 = l[0].pt with default 0.0
01:57  Adding optional observable pt_l1 = l[0].e with default 0.0
01:57  Adding optional observable eta_l1 = l[0].eta with default 0.0
01:57  Adding optional observable phi_l1 = l[0].phi() with default 0.0
01:57  Adding optional observable e_a1 = a[0].pt with default 0.0
01:57  Adding optional observable pt_a1 = a[0].e with default 0.0
01:57  Adding optional observable eta_a1 = a[0].eta with default 0.0
01:57  Adding optional observable phi_a1 = a[0].phi() with default 0.0
01:57  Adding optional observable e_j1 = j[0].pt with default 0.0
01:57  Adding optional obse

02:02    112968 / 150000 events pass everything
02:02    Deleting /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_15/tag_1_pythia8_events_delphes.root
02:02  Loading HDF5 data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/setup.h5 and saving file to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_14.h5
02:02  Weight names found in Delphes files: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
02:02  Benchmarks found in MadMiner file: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 'morphing_basis_vector_5']
02:02  Starting analysis of runs for card 15
02:02  Benchmarks found in MadMiner file: ['sm', 'morphing_basis_vector_1', 'morphing_basis_vector_2', 'morphing_basis_vector_3', 'morphing_basis_vector_4', 

02:16    First 10 values for observable m_l1_a1:
[2234.87889877           nan 3309.41628288 3781.00756917 3603.48127702
           nan 5648.61557631           nan 2615.3000868            nan]
02:16    First 10 values for observable deltaeta_l1_a1:
[ 0.80655472         nan  1.15391898  0.71492417  1.27888839         nan
 -2.20313174         nan -0.44199303         nan]
02:16    First 10 values for observable deltaphi_l1_a1:
[-3.09373461         nan -3.15202057  3.15759706  3.20213902         nan
 -3.15523344         nan -3.12088227         nan]
02:17    First 10 values for observable m_a1_l1_met:
[3834.47860996           nan 4282.00979047 3821.34730427 5642.4295143
           nan 7364.08637524           nan 3890.92677025           nan]
02:17    First 10 values for observable pt_a1_l1_met:
[   7.8663397            nan   26.58997423   11.39286687 1198.86036169
           nan 1597.90632443           nan  223.27167534           nan]
02:17    First 10 values for observable mt:
[ 63.92705828 

02:29    First 10 values for observable phi_l1:
[-2.76458693 -1.27152526 -0.70320594  0.         -1.34871125 -1.03655005
  3.01211023 -0.35141391  0.          0.7533524 ]
02:29    First 10 values for observable e_a1:
[2104.84594727 2096.93359375 2206.25683594 2105.32202148 2184.05932617
 2092.93188477 2163.59960938 2103.4140625  2256.09667969 1979.93005371]
02:29    First 10 values for observable pt_a1:
[3748.66748047 2112.65307617 2862.10424805 2924.9309082  2278.67626953
 2169.31420898 2860.91967773 2235.42553711 2294.90063477 2066.7109375 ]
02:29    First 10 values for observable eta_a1:
[-1.18009734  0.12236868  0.75313431 -0.85601127  0.29329988 -0.26935321
  0.78273028 -0.35246229 -0.1852055  -0.29500431]
02:30    First 10 values for observable phi_a1:
[ 0.41265839  1.8219744   2.52504849 -2.79367423  1.74794328  2.07924318
 -0.12044226  2.80709243 -2.49690533 -2.38317943]
02:30    First 10 values for observable e_j1:
[  86.96953583   97.00199127  134.30310059 1720.53173828   36.

02:42  Adding optional observable pt_j1 = j[0].e with default 0.0
02:42  Adding optional observable eta_j1 = j[0].eta with default 0.0
02:42  Adding optional observable phi_j1 = j[0].phi() with default 0.0
02:42  Adding required observable deltaphi_l1_met = l[0].phi() - met.phi()
02:42  Adding required observable deltaphi_a1_met = a[0].phi() - met.phi()
02:42  Adding required observable m_l1_met = (l[0] + met).m
02:42  Adding required observable pt_l1_met = (l[0] + met).pt
02:42  Adding required observable m_l1_a1 = (l[0] + a[0]).m
02:42  Adding required observable deltaeta_l1_a1 = l[0].eta - a[0].eta
02:42  Adding required observable deltaphi_l1_a1 = l[0].phi() - a[0].phi()
02:42  Adding required observable m_a1_l1_met = (a[0] + l[0] + met).m
02:42  Adding required observable pt_a1_l1_met = (a[0] + l[0] + met).pt
02:42  Adding required observable mt defined through external function
02:42  Adding required observable phi_resurrection defined through external function
02:42  Adding cut 

## Combine samples

In [13]:
filenames_in = [sample_dir + 'samples_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples.h5')

02:48  Careful: this tool assumes that all samples are generated with the same setup, including identical benchmarks (and thus morphing setup). If it is used with samples with different settings, there will be wrong results! There are no explicit cross checks in place yet.
02:48  Copying setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_0.h5 to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
02:48  Loading samples from file 1 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_0.h5
02:48  Loading samples from file 2 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_1.h5
02:48  Loading samples from file 3 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_2.h5
02:48  Loading samples from file 4 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/sam

## Main loop  with tight cuts

In [9]:
run = 0

for i_card in range(n_runs_per_benchmark):
    logging.info('Starting analysis of runs for card {}'.format(i_card))
            
    # Load setup
    dp = DelphesProcessor(sample_dir + 'setup.h5', debug=False)
    
    # Load events
    for benchmark in ['sm']:
        run += 1
        run_str = str(run)
        if len(run_str) < 2:
            run_str = '0' + run_str

        if run < 15:
            dp.add_hepmc_sample(
                mg_process_dir + 'Events/run_{}/tag_2_pythia8_events.hepmc.gz'.format(run_str),
                sampled_from_benchmark=benchmark
            )
        else:
            dp.add_hepmc_sample(
                mg_process_dir + 'Events/run_{}/tag_1_pythia8_events.hepmc.gz'.format(run_str),
                sampled_from_benchmark=benchmark
            )
        
    # Run Delphes
    dp.run_delphes(
        delphes_directory=delphes_dir,
        delphes_card=card_dir + 'delphes_card.dat',
        log_directory=log_dir,
        initial_command='source activate python2',
    )
    
    # Set up observables and cuts
    setup_observables(dp)
    setup_cuts(dp, use_tight_cuts=True)
    
    # Analyse Delphes sample
    dp.analyse_delphes_samples(delete_delphes_files=True)
        
    # Save
    dp.save(sample_dir + 'samples_tight_{}.h5'.format(i_card))


16:44  
16:44  ------------------------------------------------------------
16:44  |                                                          |
16:44  |  MadMiner v2018.11.02                                    |
16:44  |                                                          |
16:44  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
16:44  |                                                          |
16:44  ------------------------------------------------------------
16:44  
16:44  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_2_pythia8_events.hepmc.gz
16:50  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_01/tag_2_pythia8_events_delphes.root
16:57    No observations remainining!
16:57  Starting analysis of runs for card 1
16:58  Running Delphes 

22:25    102706 / 150000 events pass everything
22:25  Starting analysis of runs for card 16
22:26  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_17/tag_1_pythia8_events.hepmc.gz
22:35  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_17/tag_1_pythia8_events_delphes.root
22:41    104133 / 150000 events pass everything
22:41  Starting analysis of runs for card 17
22:42  Running Delphes (/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/Delphes) on event sample at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_18/tag_1_pythia8_events.hepmc.gz
22:50  Analysing Delphes sample /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/mg_processes/wgamma/Events/run_18/tag_1_pythia8_events_delphes.root
22:57   

## Combine samples with tight cuts

In [10]:
filenames_in = [sample_dir + 'samples_tight_{}.h5'.format(i_card) for i_card in range(n_runs_per_benchmark)]

combine_and_shuffle(filenames_in, sample_dir + 'samples_tight.h5')

22:57  Careful: this tool assumes that all samples are generated with the same setup, including identical benchmarks (and thus morphing setup). If it is used with samples with different settings, there will be wrong results! There are no explicit cross checks in place yet.
22:57  Copying setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight_0.h5 to /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight.h5
22:57  Loading samples from file 1 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight_0.h5
22:57  Loading samples from file 2 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight_1.h5
22:57  Loading samples from file 3 / 18 at /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples_tight_2.h5
22:57  Loading samples from file 4 / 18 at /Users/johannbrehmer/work/projects/ma